In [ ]:
import numpy as np
import sys, os, git
import pint
ureg = pint.UnitRegistry()

In [ ]:
current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")
sys.path.append(git_path+'/pyplot-perso')
sys.path.append(git_path+"/pyplot-perso/colors")
from colorseries import colorserie1
from plttools import PltTools

In [ ]:
T = 300 * ureg.kelvin
kB =  1.380649e-23 * ureg.joule/ureg.kelvin # J/K
Na = 6.022141e23/ureg.mol # mol-1
h = 6.626e-34 * ureg.joule*ureg.second # Js
mass_H2O_g_mol = 18.015 * ureg.gram/ureg.mol
mass_H2O = mass_H2O_g_mol / Na
mass_H2O = mass_H2O.to_base_units()
deBroglie_wavelength = np.sqrt(h**2/(2*np.pi*mass_H2O*kB*T)).to_base_units()

In [ ]:
# Theory
def density_from_mu_id(mu_K, deBroglie_wavelength, kB, T):
    mu_J = (mu_K*kB).to_base_units()
    return np.exp(-mu_J/(kB*T)) / deBroglie_wavelength**3

# Directory scanner
def fast_scandir(dirname):
    subfolders= [f.path for f in os.scandir(dirname) if f.is_dir()]
    for dirname in list(subfolders):
        subfolders.extend(fast_scandir(dirname))
    return subfolders

def measure_isotherm(dirs, atoms = ["CO2"], data_type = "density"):
    out_data = []
    for dir in dirs:
        mean_out = 0
        for atom in atoms:
            if data_type == "density":
                out = np.loadtxt(dir+"/density_"+atom+".dat")[:,1]
            elif data_type == "pressure":
                out = np.loadtxt(dir+"/density_"+atom+".dat")[:,1]
            N_end = len(out)//50
            mean_out += np.mean(out[-N_end:])
        out_data.append(mean_out)
    out_data = np.array(out_data)
    if data_type == "density":
        out_data *= ureg.gram/ureg.centimeter**3
    elif data_type == "pressure":
        out_data *= ureg.atm
    return out_data

# Folder detector
def scan_directory(dir_lammps):
    subfolders= fast_scandir("./")
    filtered_subfolders = []
    for dir in subfolders:
        if "outputs_" in dir:
            if dir_lammps:
                if "lammps-comparison" in dir:
                    filtered_subfolders.append(dir)
            else:
                if "lammps" not in dir:
                    filtered_subfolders.append(dir) 
    chemical_potentials_K = []
    for dir in filtered_subfolders:
        mu_K = np.float32(dir.split("_")[-1][2:])
        chemical_potentials_K.append(mu_K)
    chemical_potentials_K = np.array(chemical_potentials_K)
    chemical_potentials_K = chemical_potentials_K *ureg.degree_Kelvin
    return filtered_subfolders, chemical_potentials_K

# LAMMPS data

In [ ]:
lammps_dir, lammps_mu_K = scan_directory(dir_lammps = True)
lammps_rho_g_cm3 = measure_isotherm(lammps_dir, atoms = ["H2O"], data_type = "density")
lammps_p_atm = measure_isotherm(lammps_dir, atoms = ["H2O"], data_type = "pressure")

# GOMC data

In [ ]:
gomc_dir, gomc_mu_K = scan_directory(dir_lammps = False)
gomc_rho_g_cm3 = measure_isotherm(gomc_dir, atoms = ["H", "O"], data_type = "density")
gomc_p_atm = measure_isotherm(gomc_dir, atoms = ["H", "O"], data_type = "pressure")

# Theory

In [ ]:
mu_K = np.arange(3500, 5500)*ureg.kelvin
rho_m3 = density_from_mu_id(mu_K, deBroglie_wavelength, kB, T) # per m3
rho_g_cm3 = (rho_m3 * mass_H2O_g_mol / Na).to(ureg.gram/ureg.cm**3)

# Plot

In [ ]:
for dark_mode in [False, True]:

    # Initialise figure
    myplt = PltTools()
    myplt.prepare_figure(fig_size = (18,12), dark_mode = dark_mode,
                         transparency = True, n_colone = 1)
    # Panel 1
    myplt.add_panel(panel_position = [0, 0])
    myplt.add_plot(x = gomc_mu_K.magnitude*0.00198,
                   y = gomc_rho_g_cm3.magnitude,
                   #type = "semilogy",
                   marker = "o", data_color = 0,
                   markersize = 12, data_label = r"$\textrm{GOMC}$")
    myplt.add_plot(x = lammps_mu_K.magnitude*0.00198,
                   y = lammps_rho_g_cm3.magnitude,
                   #type = "semilogy",
                   marker = "s", data_color = 1,
                   markersize = 12, data_label = r"$\textrm{LAMMPS}$")
    myplt.add_plot(x = mu_K.magnitude*0.00198,
                   y = rho_g_cm3.magnitude,
                   #type = "semilogy",
                   marker = "-", data_color = 2,
                   data_label = r"$\Lambda^{-3} \exp(\mu/k_\textrm{B}T)$")
    myplt.complete_panel(xlabel = r"$-\mu ~ (\textrm{kcal/mol})$",
                         ylabel = r"$\rho ~ (\textrm{g/cm}^3)$",
                         panel_title = r"$\textrm{Bulk H$_2$O isotherm}$",
                         xpad = 10, legend=True)
    #myplt.set_boundaries(x_boundaries = (5, 11), y_boundaries = (0, 0.6))
    # Finish figure
    # myplt.add_subplotlabels(type_label_panel = "a")
    myplt.save_figure(filename = "H2O", saving_path = './')